In [1]:
import pandas as pd
from utils.task1_utils import *

In [2]:
input_file_path = r'./data/dummy_data.csv'

In [5]:
df = pd.read_csv(input_file_path, header=None, names=['uid', 'item'])
df

,uid,item
0,1,apple
1,1,carrot
2,1,durian
3,2,banana
4,2,carrot
5,2,edamame
6,3,apple
7,3,banana
8,3,carrot
9,3,edamame


In [4]:
qualifiedUsers = df.groupby('uid')['item'].apply(set).reset_index()
qualifiedUsers = list(qualifiedUsers['item'].to_dict().values())
qualifiedUsers

[{'apple', 'carrot', 'durian'},
 {'banana', 'carrot', 'edamame'},
 {'apple', 'banana', 'carrot', 'edamame'},
 {'banana', 'edamame'}]

In [6]:
def apriori(chunk, minsup):
    # chunk_list is needed here because chunk is of type TraversableOnce, which will be empty after calling list
    # chunk = list(chunk)
    
    if minsup < 1:
        minsup = 1

    # contains all k-tuple itemsets as result
    result_candidate_itemsets = list()  
    
    # contains the true frequent itemsets used to generate candidate for next k in apriori
    true_frequent_k_itemset_list = list()
    
    # contains candidate of next k-tuple itemsets, used to decide if there are any candidate left
    next_k_candidate_list = list()
    k_index = 1

    # generate true frequent singletons
    # true_frequent_k_itemset_list is now the frequent singletons
    # next_k_candidate_list is now the singleton candidates

    true_frequent_k_itemset_list, next_k_candidate_list = generateFrequentSingletons(
        chunk, minsup)

    while len(next_k_candidate_list) != 0:
        k_index += 1  # generate 2-tuple candidates, 3-tuple, 4-tuple, etc

        if k_index == 2:

            result_candidate_itemsets.append(
                # append candidate k
                [(single,) for single in true_frequent_k_itemset_list])
            next_k_candidate_list = generateKTupleItemsets(
                # true k-1 to candidate k
                [(single,) for single in true_frequent_k_itemset_list], 2)
        else:
            true_frequent_k_itemset_list = countFrequentItemsets(
                next_k_candidate_list, chunk, minsup)  # cand k to true k
            result_candidate_itemsets.append(
                true_frequent_k_itemset_list)  # append candidate k
            next_k_candidate_list = generateKTupleItemsets(
                true_frequent_k_itemset_list, k_index)  # true k to candidate k+1

    return result_candidate_itemsets

In [7]:
%%time

fi = apriori(qualifiedUsers, minsup=3)

CPU times: user 58 µs, sys: 24 µs, total: 82 µs
Wall time: 264 µs


In [8]:
fi

[[('carrot',), ('banana',), ('edamame',)], [('banana', 'edamame')]]